# This notebook

Note: installing neuron was not that straightforward.





In [48]:
import argparse
import numpy as np
import datetime
import pickle 
import warnings
import neuron
import sys
#! echo $PYTHONPATH

ImportError: No module named 'neuron'

In [ ]:
a = Brep()
#a.parser.set_defaults()
#print (a.args.aa_length)
a.parser.add_argument('ba', type = int, default = 1)
a.args = a.parser.convert_arg_line_to_args(' --verbose 1')

### Parsing the Parameter file
Not nice: there are about 4 different names for one variable- there are the default ones in the code, the assigned ones in the code, and the ones in the parameter file. 
I will use variables that are similar to the ones defined in the grammar in the code, with the difference that they will be adapted to Python syntax ( _ instead of -), and a few have an additional postfix to clarify what they do (e.g. \_fn = filename) 


### On the installation of neuron
Neuron did not really work out of the box for me. 
I (ubuntu 16.04 LTS, 64 bit) did it the following way:
- Download the .rpm package from here: https://www.neuron.yale.edu/neuron/download
- Install it with: 
    `alien -i nrn_...*package*` (Note that the .deb package did not work out, and neither did the installation using rpm directly)
- Edit the .bashrc file by adding the following lines: 

    `#Added for neuron
    export PYTHONPATH="${PYTHONPATH}:/usr/lol/nrn/lib/python/" `
    
    (first check that this path is actually where it got installed by going to the folder and see whether `python -c 'import neuron'` tells you about your NEURON version or whether there ain't no module called neuron.


http://www.davison.webfactional.com/notes/hoc-to-python-bulbnet/

In [43]:
b = Brep()
b.init_from_script(['--config_fn','./input_files/Parameters.hoc'])
print (b.cl_args)

0
1
{'config_fn': './input_files/Parameters.hoc'}


In [44]:
from neuron import hoc, h
# weird thing that I did not get yet: despite all the copy statements, the second time you calculate d_l, it would give 0. 
# Thus, the hoc objects must somehow take on each other's parameters... 

#load an empty hoc object and find out which parameters are native to that object (probably useless...)
empty_hoc = dir(hoc.HocObject()).copy()
config_fn = './input_files/Parameters.hoc'
#load our own hoc object from the parameter file, get the disjunct list of parameters (probably useless...)
h.xopen(config_fn)
full_hoc = dir(h).copy()
if 'd_l' not in globals():
    d_l = list (set (full_hoc)  - set (empty_hoc)).copy()
    
c_d = b.config_dict #this dict translates the parameters used in the Parameters file to the ones used in the code
# Check if the Brep object contains the right parameters and if so, change them.
# Note: Resolve conflicts with the command line - I think default should be that command line should has priority 
for h_k in d_l:
    if hasattr (self.args, c_d[h_k]):
        setattr (self.args, c_d[h_k], getattr (h, h_k))
# The following two parameters are an exception:
if 'GLdepth' in d_l and 'PCLdepth' in d_l:
    setattr (self.args, 'aa-length', getattr(h, 'GLdepth')+getattr(h,'PCLdepth'))
    


ImportError: No module named 'neuron'

In [35]:
import argparse
import numpy as np
import datetime
import pickle


# On the naming conventions: I tried to keep variable names similar as the ones in the brep.scm file
# However, all - had to bee transformed to _ , and sometimes a postfix like _fn (filename) or _num (number) was added for clarification.

class Brep (object):
    
    def __init__(self):
        
        self.parser = argparse.ArgumentParser(prog='BREP for py ', conflict_handler='resolve')
        self.parser.add_argument("--config_fn",   type = str, default = '', help ="use the given hoc configuration file to obtain parameters")
        self.parser.add_argument("--gc_points_fn",  type = str, default = '', help ="load originating points for granule cells from given file, default is randomly generated")
        self.parser.add_argument("--gct_points_fn", type = str, default = '', help ="load junction points for parallel fibers from given file, default is offset from GC soma points")
        self.parser.add_argument("--goc_points_fn", type = str, default = '' , help ="load originating points for Golgi cells from given file, default is randomly generated") #!!
        self.parser.add_argument("--rng_seeds",     type =list, default = [13, 17, 19, 23, 29, 37], help ="Use the given seeds for random number generation")
        self.parser.add_argument("--mpi_split",     type = int, default = 0, help ="perform MPI split operation") #actually it should be color, and only if this element exists, the communicator is split. Thus, not sure if I put the right default.
        
        self.parser.add_argument("--x_extent",  type = float, default = 1200.0, help ="X_extent of patch")
        self.parser.add_argument("--y_extent",  type = float, default = 300.0 , help ="Y_extent of patch")
        self.parser.add_argument("--z_extent",  type = float, default = 150.0 , help ="Z_extent of patch")
                                 
        self.parser.add_argument("--pf_length", type = float, default = 100.0, help ="parallel fiber length")
        self.parser.add_argument("--pf_step",   type = float, default = 30.0 , help ="parallel fiber step size")
        self.parser.add_argument("--pf_start",  type = int,   default = 0,     help ="starting index for parallel fiber points")                                     
        self.parser.add_argument("--aa_length", type = float, default = 200.0, help ="ascending axon length")
        self.parser.add_argument("--aa_step",   type = float, default = 100.0, help ="ascending axon step size")

        self.parser.add_argument("--goc_num", type = int , default = 200, help ="number of Golgi cells")
        self.parser.add_argument("--goc_mean_distance", type = float , default = 50.0, help ="mean distance between Golgi cells and Golgi cell grid")
        self.parser.add_argument("--gc_num",    type = int,   default = 10000, help ="number of granule cells and parallel fibers")
                                 
        self.parser.add_argument("--goc_start",        type = int,   default = 0,    help ="starting index for Golgi cell points default is 0")
        self.parser.add_argument("--pf_goc_zone",      type = float, default = 5.0,  help ="PF to Golgi cell connectivity zone" ) # verify datatype, not sure what the zone parameters mean so far.
        self.parser.add_argument("--aa_goc_zone",      type = float, default = 5.0,  help ="AA to Golgi cell connectivity zone" )  
        self.parser.add_argument("--goc_goc_zone",     type = float, default = 30.0, help ="Golgi to Golgi cell connectivity zone" )
        self.parser.add_argument("--goc_goc_gap_zone", type = float, default = 30.0, help ="Golgi to Golgi gap junction connectivity zone")
                                 
        #GOC dendrites
        self.parser.add_argument("--goc_dendrites",               type = int,   default = 4,     help ="number of Golgi cell dendrites")                                 
        
        self.parser.add_argument("--goc_theta_apical_min",        type = float, default = 30.0,  help ="min angle used to determine height of apical dendrite in z direction")
        self.parser.add_argument("--goc_theta_apical_max",        type = float, default = 60.0,  help ="max angle used to determine height of apical dendrite in z direction")
        self.parser.add_argument("--goc_theta_apical_stdev",      type = float, default = 1.0,   help ="stdev of angle used to determine height of apical dendrite in z direction")
        
        self.parser.add_argument("--goc_theta_basolateral_min",   type = float, default = 30.0,  help ="min angle used to determine height of basolateral dendrite in z direction")
        self.parser.add_argument("--goc_theta_basolateral_max",   type = float, default = 60.0,  help ="max angle used to determine height of basolateral dendrite in z direction")
        self.parser.add_argument("--goc_theta_basolateral_stdev", type = float, default = 1.0,   help ="stdev of angle used to determine height of basolateral dendrite in z direction")
        
        self.parser.add_argument("--goc_apical_dendheight",       type = float, default = 100.0, help ="height of Golgi cell apical dendritic cone")
        self.parser.add_argument("--goc_apical_radius",           type = float, default = 100.0, help ="radius of Golgi cell apical dendrite cone")
        self.parser.add_argument("--goc_apical_nseg",             type = int,   default = 2,     help ="number of segments of GoC apical dendrite")
        self.parser.add_argument("--goc_apical_nsegpts",          type = int,   default = 4,     help ="number of points per segments of GoC apical dendrite")
        
        self.parser.add_argument("--goc_basolateral_dendheight",  type = float, default = 100.0, help ="height of Golgi cell basolateral dendritic cone")
        self.parser.add_argument("--goc_basolateral_radius",      type = float, default = 100.0, help ="radius of Golgi cell basolateral dendrite cone")
        self.parser.add_argument("--goc_basolateral_nseg",        type = int,   default = 2,     help ="number of segments of GoC basolateral dendrite")
        self.parser.add_argument("--goc_basolateral_nsegpts",     type = int,   default = 4,     help ="number of points per segments of GoC basolateral dendrite")
        
        #GOC axons
        self.parser.add_argument("--goc_axons",      type = int,   default = 10,    help ="number of Golgi cell axons")
        self.parser.add_argument("--goc_axonsegs",   type = int,   default = 1,     help ="number of Golgi cell axon segments")
        self.parser.add_argument("--goc_axonpts",    type = int,   default = 2,     help ="number of Golgi cell axon points")
        self.parser.add_argument("--goc_axon_x_min", type = float, default =-200.0, help ="minimum extent of Golgi cell axons along X axis")
        self.parser.add_argument("--goc_axon_x_max", type = float, default = 200.0, help ="maximum extent of Golgi cell axons along X axis")
        self.parser.add_argument("--goc_axon_y_min", type = float, default =-200.0, help ="minimum extent of Golgi cell axons along Y axis")
        self.parser.add_argument("--goc_axon_y_max", type = float, default = 200.0, help ="maximum extent of Golgi cell axons along Y axis")
        self.parser.add_argument("--goc_axon_z_min", type = float, default =-30.0,  help ="minimum extent of Golgi cell axons along Z axis")
        self.parser.add_argument("--goc_axon_z_max", type = float, default =-200.0, help ="maximum extent of Golgi cell axons along Z axis")
        
        #Output utilities
        self.parser.add_argument("--save_aa", type = bool, default = False, help ="save ascending axon points")
        self.parser.add_argument("--save_pf", type = bool, default = False, help ="save parallel fiber points")
        self.parser.add_argument("--output",  type = str,  default = '',    help ="specify output file prefix, default is a timestemp.") # we should set a default here that puts in a timeprint in order to prevent unvoluntary overwrite
        self.parser.add_argument("--verbose", type = bool, default = False, help ="verbose model")
        self.config_dict = {
        'y-extent' : 'GoCyrange', 
        'goc-apical-nsegpts' : 'GoC_Ad_nsegpts', 
        'goc-theta-basolateral-min' : 'GoC_Btheta_min', 
        'goc-basolateral-nseg' : 'GoC_Bd_nseg', 
        'goc-basolateral-nsegpts' : 'GoC_Bd_nsegpts', 
        'goc-axon-z-min' : 'GoC_Axon_Zmin', 
        'goc-axon-y-max' : 'GoC_Axon_Ymax', 
        'num-gc' : 'numGC', 
        'goc-basolateral-dendheight' : 'GoC_PhysBasolateralDendH', 
        'aa-goc-zone' : 'AAtoGoCzone', 
        'mean-goc-distance' : 'meanGoCdistance', 
        'goc-theta-apical-min' : 'GoC_Atheta_min', 
        'goc-dendrites' : 'numDendGolgi', 
        'goc-axon-x-min' : 'GoC_Axon_Xmin', 
        'goc-theta-basolateral-stdev' : 'GoC_Btheta_stdev', 
        'goc-theta-apical-stdev' : 'GoC_Atheta_stdev', 
        'goc-theta-apical-max' : 'GoC_Atheta_max', 
        'goc-axonsegs' : 'GoC_Axon_nseg', 
        'num-goc' : 'numGoC', 
        'goc-axonpts' : 'GoC_Axon_npts', 
        'z-extent' : 'GoCzrange', 
        'goc-axons' : 'numAxonGolgi', 
        'goc-theta-basolateral-max' : 'GoC_Btheta_max', 
        'pf-step' : 'PFstep', 
        'aa-step' : 'AAstep', 
        'x-extent' : 'GoCxrange', 
        'goc-apical-radius' : 'GoC_PhysApicalDendR', 
        'goc-axon-x-max' : 'GoC_Axon_Xmax', 
        'goc-basolateral-radius' : 'GoC_PhysBasolateralDendR', 
        'goc-goc-gap-zone' : 'GoCtoGoCgapzone', 
        'goc-apical-nseg' : 'GoC_Ad_nseg', 
        'goc-axon-y-min' : 'GoC_Axon_Ymin', 
        'goc-axon-z-max' : 'GoC_Axon_Zmax', 
        'goc-apical-dendheight' : 'GoC_PhysApicalDendH', 
        'goc-goc-zone' : 'GoCtoGoCzone', 
        'pf-goc-zone' : 'PFtoGoCzone', 
        'pf-length' : 'PFlength' }

        
    def init_from_script (self, cml_str = ['']):
        self.args = self.parser.parse_args(cml_str)
        d  =  {}
        for k_c in range(len(cml_str)-1):
            print (k_c)
            if len(str(cml_str[k_c])) > 2:
                print ('1')
                if cml_str[k_c][:2] == '--':
                    d[cml_str[k_c][2:]] = cml_str[k_c+1]
        setattr (self, 'cl_args', d)
                
            

        
    def init_from_cl(self):
        self.args = self.parser.parse_args()
        from collections import defaultdict
        d=defaultdict(list)
        for k, v in ((k.lstrip('-'), v) for k,v in (a.split('=') for a in sys.argv[1:])):
            d[k].append(v)
        setattr (self, 'cl_args', d)
        #https://stackoverflow.com/questions/12807539/how-do-you-convert-command-line-args-in-python-to-a-dictionary
        
    def check_output_prefix(self):
        '''checks if there is a specified output prefix.
        If not, will generate one from the timestampe'''
        if self.args.output == '':
            ! mkdir 'Res_{:%Y-%m-%d_%H-%M-%S}'.format(datetime.datetime.now())
            self.args.output = 'Res_{:%Y-%m-%d_%H-%M-%S}/'.format(datetime.datetime.now())
            
    def read_in_config(self):
        '''checks if a config file has been specified, and if so, updates the args
        Warns if the file cannot be found, opened or properly read in (e.g. unkown parameter) '''
        if self.args.config == '':
            warnings.warn('Cannot find config file!')
            
    
    def p_verb (stat, *args):
        '''prints statement only if the print mode is on.
        Prints args in some smart ways'''
        pass
    
    def get_GC_Points():
        ''' 
        corresponds to GC-Points statement in brep.csm file.
        Read in the GC-Points provided in a file or render some randomly. (UniformRandomProcess)
        '''
        pass
        
    def get_GCT_Points():
        '''
        corresponds to the GCT-Points statement from the 
        '''
    
    def get_GOC_Points():
        '''
        get GOC Points from file or render them
        '''
        


### Checking out the kd-tree libraries

Scipy:
- Documentation: https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.KDTree.html
- Source Code: https://github.com/scipy/scipy/blob/master/scipy/spatial/kdtree.py
- Algorithm reference Maneewongvatana and Mount 1999
- Can be queried for r nearest neighbors, however r should be relatively small because elsewise, brute force is just as efficient.
- Approximate nearest neighbors seems to be another, and much faster option, and might work well for us.
- Uses pythonic libraries. 
- The heap queue algorithm: https://github.com/python/cpython/blob/2.7/Lib/heapq.py seems to be used, but it is also in python


SKlearn:
- Documentation: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html
- Source Code: https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/neighbors/kd_tree.pyx
- uses cython. Might thus be faster. Let's check.


Neither of them appears to have parallelization supported right from the beginning.
Both just take regular arrays as inputs.


Could be interesting:
http://ieeexplore.ieee.org/abstract/document/5654017/?reload=true
GPU implementation for kNN search
Following this:
https://link.springer.com/chapter/10.1007/978-3-642-38628-2_67



# Random and deprecated stuff below this.

In [41]:
### The transformations of the parameters, automized in a small parser script.

#Had to be done only once, but will be kept for reference.

from neuron import hoc, h
# weird thing that I did not get yet: despite all the copy statements, the second time you calculate d_l, it would give 0. 
# Thus, the hoc objects must somehow take on each other's parameters... 

#load an empty hoc object and find out which parameters are native to that object (probably useless...)
empty_hoc = dir(hoc.HocObject()).copy()
config_fn = './input_files/Parameters.hoc'
#load our own hoc object from the parameter file, get the disjunct list of parameters (probably useless...)
h.xopen(config_fn)
full_hoc = dir(h).copy()
if 'd_l' not in globals():
    d_l = list (set (full_hoc)  - set (empty_hoc)).copy()

# Code file
tf = 'brep_commented.scm'
# Step one: parse all lines that contain both config or options as those are the ones that 
res_dict = {}
with open (tf, 'rb') as tff:
    for line in tff:
        st = str(line)
        c = st.find ("config '")
        o = st.find ("options '") 
        if c > 0 and o > 0:
            c_clb = st[c:].find (')')
            o_clb = st[o:].find (')')
            res_dict[st[c+8:c+c_clb]] = st[o+9:o+o_clb] 
    tff.close()

#parameters that are defined in the res file but have not been parsed yet
rest_hk = (set (d_l)- set(res_dict.keys()))
rem = {}
#check if they occur in the code
with open ('brep_commented.scm', 'rb') as f_in:
    n = 0
    for line in f_in:
        n = n+1
        for w in rest_hk:
            if str(line).find(w)>0:
                if w in rem.keys():
                    rem[w].append(n)
                else:
                    rem[w] = [n]                  
print (rem) # 'TS is coincidental, the other two parameters are taken seperate care of.

#print the resulting dict to a file.
with open ('par_d2.txt', 'w') as f_out:
    for k in res_dict.keys():
        f_out.write("'" +res_dict[k]+ "' : '"+k+ "', \n" )

#print (res_dict)

ImportError: No module named 'neuron'

In [5]:
#Used this to try out command line calls. 

#Brep = importlib.reload(BREPpy)
#stupid workaround so that the known command line call can be kept up.
#I think I changed something, would have to git checkout....
class Brep2 (Brep, b):
    def __init__:
        self.args = b.args
        self.config_dict = b.config_dict
        self.cl_args = b.cl_args
        


def new_Brep (arg_dict = {}, **kwargs):
    if True: #delete and make new Brep file
        ! python ~/Desktop/LabRot_OIST/pybrep/BREPpy.py --config_file blabla 
        a = pkl.load(open('./tmp.pkl', 'rb'))
        ! rm tmp.pkl
        b = Brep2(a)
    else: b = Brep2(pkl.load(open('./tmp.pkl', 'rb'))) 
    #Process and add arguments    
    arg_dict.update(kwargs)
    for k in arg_dict.keys():
        if hasattr (b.args, k):
            setattr (b.args, k, arg_dict[k])
        else:
            warnings.warn ('Keyline argument {} not known'.format(k))
            
    return b

arg_dict = {'config_file': 'blabla.c',
            'verbose': True}
b = new_Brep(arg_dict, gc_points_fn = 'yipyip' )


I
Traceback (most recent call last):
  File "/home/ines/Desktop/LabRot_OIST/pybrep/BREPpy.py", line 134, in <module>
    Brep().initialize_and_dump()
  File "/home/ines/Desktop/LabRot_OIST/pybrep/BREPpy.py", line 123, in initialize_and_dump
    self.cl_args = self.parser.convert_arg_line_to_args()
TypeError: convert_arg_line_to_args() missing 1 required positional argument: 'arg_line'


FileNotFoundError: [Errno 2] No such file or directory: './tmp.pkl'

In [52]:
## Fun with magic

# http://ipython.readthedocs.io/en/stable/interactive/magics.html
#notable ones
# %debug #-> debug stuff. Lets you inspect the stack frame of an exception interactively
# %env #-> see all env variables

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-52-e95a52b59324>, line 7)

In [58]:
%run BREPpy.py

I


TypeError: convert_arg_line_to_args() missing 1 required positional argument: 'arg_line'